## A) Descriptive analysis
1. Please describe the format of the data files. Can you identify any limitations or distortions of the data?
2. What is the most popular name of all time? (Of either gender.)
3. What is the most gender ambiguous name in 2013? 1945?
4. Of the names represented in the data, find the name that has had the largest percentage increase in popularity since 1980. Largest decrease?
5. Can you identify names that may have had an even larger increase or decrease in popularity?

#### Part 1 - Understanding the file

- Q: Please describe the format of the data files. Can you identify any limitations or distortions of the data?

- Data Format:

    - The data is provided in comma-delimited text files (.TXT), one for each state.

    - Each record contains five fields: State Code, Sex (M/F), Year of Birth, Name, and the number of occurrences.

- Data Limitations & Distortions:

    1. Exclusion of Uncommon Names: The most significant limitation is that names with fewer than 5 occurrences in a given state and year are completely omitted. This distorts the data by over-representing popular names and making it impossible to study the "long tail" of rare or unique names.

    2. Inaccurate National Aggregates: As a direct result of the first point, summing the state data will not produce an accurate national count. The true national total for any name will be higher than what can be calculated from these files.

    3. Source-Related Bias: The data is sourced exclusively from Social Security records. This means individuals born before the widespread adoption of Social Security in the mid-1930s, or those who never received a Social Security Number for other reasons, are not included. This particularly impacts the reliability of the data for the earliest years in the dataset.

#### Part 2 - Searching through the data

- What is the most popular name of all time? (Of either gender.)

In [1]:
#### Import necessary libraries
import os
import glob             # Finds all the files ending with .txt
import pandas as pd
import numpy as np
from dotenv import load_dotenv


In [2]:
# Load variables from the .env file
load_dotenv()

# Get the data path from the environment variables
path = os.getenv("DATA_PATH")

# Check if the path was loaded correctly
if not path:
    raise ValueError("DATA_PATH was not found!")

# Locate the files ending in .txt
all_files = glob.glob(path + "/*.TXT")

# Create an empty list for all the dataframes
df_list = []

# Provide the column names since the text files do not
column_names = ['State','Gender','Birth_year','Name','Name_occurrence']

# loop through all the text files and create a dataframe for each
# Add each dataframe to the df_list
for filename in all_files:
    df = pd.read_csv(filename, header=None, names=column_names)
    df_list.append(df)

# Add dataframes together along the row (axis=0)
# Show the dataframe
names_df = pd.concat(df_list, axis=0, ignore_index=True)
print("Data was successfully loaded!")
names_df


Data was successfully loaded!


,State,Gender,Birth_year,Name,Name_occurrence
0,IN,F,1910,Mary,619
1,IN,F,1910,Helen,324
2,IN,F,1910,Ruth,238
3,IN,F,1910,Dorothy,215
4,IN,F,1910,Mildred,200
...,...,...,...,...,...
6311499,DE,M,2021,Thiago,5
6311500,DE,M,2021,Travis,5
6311501,DE,M,2021,Troy,5
6311502,DE,M,2021,Walker,5


In [3]:
# Groupby name and sum the occurrences each time it is found
# Sort the names in descending order to determine max value
total_name_counts = names_df.groupby('Name')['Name_occurrence'].sum()
most_popular_name = total_name_counts.sort_values(ascending=False).index[0]
print(f'The most popular name of all time is: {most_popular_name} with a total count of {total_name_counts.loc[most_popular_name]}')

The most popular name of all time is: James with a total count of 5054074


In [4]:
total_name_counts

Name
Aaban         12
Aadam          6
Aadan         23
Aadarsh       11
Aaden       4173
            ... 
Zyria         81
Zyriah        63
Zyron          5
Zyrus          5
Zyshonne       5
Name: Name_occurrence, Length: 32403, dtype: int64

#### Part 3 - Boy or Girl
- Q: What is the most gender ambiguous name in 2013? 1945?

In [5]:
# Function to isolate the names by a specific year
def names_by_year(dataframe, column_name, year):
    """
    Filters a DataFrame to select rows for a specific year.

    Args:
        dataframe (pd.DataFrame): The input DataFrame.
        column_name (str): The name of the column containing the year.
        year (int): The year to filter by.

    Returns:
        pd.DataFrame: A new DataFrame containing only the data for the specified year.
    """
    names_in_year = dataframe[dataframe[column_name] == year]
    return names_in_year

# Function to calculate the total for each name in a specific year
def get_name_totals(dataframe_year, name_col, sex_col, count_col):
    """
    Groups a DataFrame by name and sex to calculate the total count for each name.

    Args:
        dataframe_year (pd.DataFrame): The input DataFrame (e.g., names_2013).
        name_col (str): The column name for the names.
        sex_col (str): The column name for the sex/gender.
        count_col (str): The column name for the counts/occurrences.

    Returns:
        pd.DataFrame: A new DataFrame with the summed counts for each name/sex combination.
    """
    # Group by the name and sex columns, then sum the count column for each group.
    name_totals = dataframe_year.groupby([name_col, sex_col])[count_col].sum().reset_index()
    return name_totals

# Function to calculate the ambiguity score of the names in the dataframe
def calculate_ambiguity_score(dataframe_name, female_col='F', male_col='M'):
    """
    Calculates and adds Total, Min_count, and Ambiguity_score columns to a DataFrame.

    Args:
        dataframe_name (pd.DataFrame): The input DataFrame. Must have columns for female and male counts.
        female_col (str): The name of the column for female counts. Defaults to 'F'.
        male_col (str): The name of the column for male counts. Defaults to 'M'.

    Returns:
        pd.DataFrame: The DataFrame with the new columns added.
    """
    # Create a copy to avoid modifying the original DataFrame unexpectedly
    df_copy = dataframe_name.copy()

    # Calculate the total and minimum counts
    df_copy['Total'] = df_copy[female_col] + df_copy[male_col]
    df_copy['Min_count'] = df_copy[[female_col, male_col]].min(axis=1)

    # Calculate the ambiguity score
    df_copy['Ambiguity_score'] = df_copy['Min_count'] / df_copy['Total']

    return df_copy

In [6]:
# Use function to isolate the names by year
names_2013 = names_by_year(names_df, 'Birth_year', 2013)
names_1945 = names_by_year(names_df, 'Birth_year', 1945)

# Use function to determine the total count for each name
names_total_2013 = get_name_totals(names_2013, 'Name','Gender', 'Name_occurrence')
names_total_1945 = get_name_totals(names_1945, 'Name','Gender', 'Name_occurrence')

In [7]:
# Create pivot table to place genders in separate columns
ambiguity_df_2013 = names_total_2013.pivot_table(
    index='Name',
    columns='Gender',
    values='Name_occurrence',
    fill_value=0 # Replace NaN with a 0
)

ambiguity_df_1945 = names_total_1945.pivot_table(
    index='Name',
    columns='Gender',
    values='Name_occurrence',
    fill_value=0 # Replace NaN with a 0 for names that are exclusively M or F
)

In [8]:
# Check if NaN or missing values were replaced with a zero
ambiguity_df_2013.isnull().sum()

Gender
F    0
M    0
dtype: int64

In [9]:
# Use the function to determine the ambiguity score of each name for the desired year
ambiguity_score_1945_df = calculate_ambiguity_score(ambiguity_df_1945, 'F', 'M')
ambiguity_score_2013_df = calculate_ambiguity_score(ambiguity_df_2013, 'F', 'M')

# Sort the rows in descending order and pull the top 10 names
ten_most_ambiguous_names_2013 = ambiguity_score_2013_df.sort_values(by=['Ambiguity_score'], ascending=False).head(10)
ten_most_ambiguous_names_1945 = ambiguity_score_1945_df.sort_values(by=['Ambiguity_score'], ascending=False).head(10)

# Print the names with the highest ambiguity score for 2013 and 1945
print(f'The most ambiguous name in the year 1945 was {ten_most_ambiguous_names_1945.index[0]}!\n')
print(f'The most ambiguous name in the year 2013 was {ten_most_ambiguous_names_2013.index[0]}!')

The most ambiguous name in the year 1945 was Maxie!

The most ambiguous name in the year 2013 was Arlin!


#### Part 4 - Mr & Ms popular

- Q: Of the names represented in the data, find the name that has had the largest percentage increase in popularity since 1980. Largest decrease?

In [10]:
# Get the total births for each year
total_births_per_year = names_df.groupby(['Birth_year'])['Name_occurrence'].sum().reset_index()
total_births_per_year.rename({'Name_occurrence': 'Total_births'}, axis=1, inplace=True)

# Merge total births back into the main dataframe
names_df = pd.merge(names_df,total_births_per_year,on='Birth_year')

"""
This allows us to compare the number of times a name appeared vs the number of births that year
"""

# Calculate the popularity of each name
names_df['Popularity'] = (names_df['Name_occurrence'] / names_df['Total_births'])

# Sum the popularity of name for each year regardless of gender
name_popularity = names_df.groupby(['Birth_year','Name'])['Popularity'].sum().reset_index()

In [13]:
# Sum the popularity for each name. This is exclusive of the gender
name_popularity = names_df.groupby(['Birth_year','Name'])['Popularity'].sum().reset_index()
name_popularity

,Birth_year,Name,Popularity
0,1910,Aaron,0.000215
1,1910,Abbie,0.000054
2,1910,Abe,0.000060
3,1910,Abner,0.000023
4,1910,Abraham,0.000267
...,...,...,...
612485,2021,Zylo,0.000002
612486,2021,Zymir,0.000034
612487,2021,Zyon,0.000072
612488,2021,Zyra,0.000013
